# ParameterTuning - XGBoost

'XGBoost (Extreme Gradient Boosting)' 는 앙상블의 부스팅 기법의 한 종류입니다. 이전 모델의 오류를 순차적으로 보완해나가는 방식으로 모델을 형성하는데, 더 자세히 알아보자면, 이전 모델에서의 실제값과 예측값의 오차(loss)를 훈련데이터 투입하고 gradient를 이용하여 오류를 보완하는 방식을 사용함.

### gridsearchCV : 최적 파라미터 선정법 찾기 
##### 각 파라미터로 여러 모델학습을 진행하여 최적의 파라미터를 찾아주는 CV함수이며 설정한 스코어에 따른 파라미터 선정한 뒤 검증

In [17]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import math
import gc
import pprint
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [18]:
train=pd.read_csv("Train.csv")
test=pd.read_csv("Test.csv")

In [19]:
train=train.drop('inst_id',axis=1)
train=train.drop('instkind_nan',axis=1)

test=test.drop('inst_id',axis=1)
test=test.drop('instkind_nan',axis=1)

train['ownerChange'] = round(train['ownerChange'])
train['bedCount_class'] = round(train['bedCount_class'])

test['ownerChange'] = round(train['ownerChange'])
test['bedCount_class'] = round(train['bedCount_class'])

train.head(5)

,OC,sido_choongchung,sido_gangwon,sido_gyeongin,sido_gyeongsang,sido_jeonra,sido_seoul,instkind_clinic,instkind_general_hospital,instkind_hospital,instkind_nursing_hospital,instkind_traditional_clinic,instkind_traditional_hospital,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,ctax1,profit1,liquidAsset1,quickAsset1,receivableS1,inventoryAsset1,nonCAsset1,tanAsset1,OnonCAsset1,receivableL1,debt1,liquidLiabilities1,shortLoan1,NCLiabilities1,longLoan1,netAsset1,surplus1,revenue2,salescost2,sga2,salary2,noi2,noe2,interest2,ctax2,profit2,liquidAsset2,quickAsset2,receivableS2,inventoryAsset2,nonCAsset2,tanAsset2,OnonCAsset2,receivableL2,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange,bedCount_class
0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,22.162515,0.000000,22.099796,21.433189,16.566138,16.539187,16.398469,17.269990,19.232365,20.735886,20.720935,19.968362,16.525413,21.645374,21.582217,18.781494,0.0,20.341751,19.120425,0.000000,19.992422,19.676297,21.768639,21.126324,22.181380,0.000000,22.123814,21.447809,16.600193,17.216153,16.568091,16.749797,19.152229,20.537140,20.520624,20.076434,16.425505,21.658620,21.592994,18.798422,0.0,20.447504,19.222130,0.000000,20.099863,19.781657,21.686169,20.963246,4.143135,4.174387,0.0,3.0
1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,23.980280,9.659782,23.481393,19.521179,17.056168,16.823799,15.443340,15.495813,20.342724,19.074269,19.037270,10.917985,16.218383,19.670926,19.568364,15.878952,0.0,19.623979,19.023845,9.354666,18.934266,18.096803,18.567912,12.280544,20.078340,9.609001,19.910191,19.440285,16.978333,17.291624,15.026843,16.011168,17.025473,19.100225,19.065542,11.998936,16.044684,19.499605,19.428651,15.372413,0.0,19.505616,18.841626,9.180719,18.812726,17.568559,18.838180,12.258481,6.687109,6.701960,0.0,4.0
2,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,20.727778,20.060616,19.918561,19.507228,11.240552,10.308986,0.000000,0.000000,17.549955,19.422937,19.351593,15.906875,16.747236,18.607003,18.607003,0.000000,0.0,18.341793,18.341793,0.000000,0.000000,0.000000,19.520988,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.459586,0.693147,0.0,4.0
3,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,25.006954,0.000000,24.981392,24.182293,20.042488,20.954034,20.902985,18.973242,20.621045,23.291406,23.168630,0.000000,21.133250,24.488628,24.369258,22.095768,0.0,24.469560,24.055180,23.713241,23.388542,22.704687,23.352064,22.880092,24.925842,0.000000,24.896486,24.114795,19.982624,21.026087,20.968110,19.204088,20.546991,23.132525,23.014845,0.000000,20.934483,24.401680,24.274484,22.088313,0.0,24.354384,23.557572,22.944579,23.755157,23.437961,23.284691,22.769775,6.498282,6.498282,0.0,4.0
4,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,24.615974,0.000000,24.587275,23.920337,18.537150,21.073281,20.991191,0.000000,18.206865,22.566524,22.493677,22.134087,19.910926,24.499866,24.491560,19.219718,0.0,24.633249,24.087425,23.609762,23.767260,23.325956,18.273475,22.923325,24.596190,0.000000,24.576086,23.878565,20.208680,21.143362,21.023260,0.000000,17.301128,22.313883,22.219315,21.936761,19.908536,24.563393,24.543819,20.486709,0.0,24.663538,24.126880,23.590810,23.784786,23.370010,15.531458,22.923325,5.332719,5.288267,0.0,3.0


In [20]:
train_lr = pd.read_csv('Train_lr.csv')

train_lr['ownerChange'] = round(train_lr['ownerChange'])
train_lr['bedCount_class'] = round(train_lr['bedCount_class'])

train_dt = pd.read_csv('Train_dt.csv')
train_dt.drop('instkind_nan', axis=1, inplace=True)

In [28]:
X = train.drop('OC', axis=1)
y = train['OC']

from sklearn.model_selection import train_test_split

train_X, resid_X, train_y, resid_y = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=22)
valid_X, test_X, valid_y, test_y = train_test_split(resid_X, resid_y, test_size=0.5, shuffle=True, random_state=22)

In [23]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [9]:
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance ## Feature Importance를 불러오기 위함

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
import warnings

참고 : https://m.blog.naver.com/gustn3964/221431933811

In [13]:
#필요한 라이브러리 임포트
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

#XGBoost 정의
#파라미터는 이전 글을 참고하여 grid 설정(dict 형태)
xgb= XGBClassifier()

# 정확도

In [32]:
xgb_param_grid={
    'n_estimators' : [100,200,300,400,500],
    'learning_rate' : [0.01,0.05,0.1,0.15],
    'max_depth' : [3,5,7,10,15],
    'gamma' : [0,1,2,3],
    'colsample_bytree' : [0.8,0.9],
    
}

#여기서 설정 파라미터의 갯수(총 4000개의 조합이므로 4000번의 학습이 돌아감)
xgb_grid=GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="accuracy", n_jobs=-1, verbose = 2)
xgb_grid.fit(train_X, train_y)

Fitting 5 folds for each of 800 candidates, totalling 4000 fits


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_to_...
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 0.9], 'gamma': [0, 1, 2, 3],
                         'learning_rate': [0.01, 0.05, 0.1, 0.15],
                         'max_depth': [3, 5, 7, 10, 15],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='accuracy', verbose=2)

In [33]:
#best acc 수치와 best parameter확인
print("best acc : {0: .4f}".format(xgb_grid.best_score_))
print("best param : ",xgb_grid.best_params_)

best acc :  0.9571
best param :  {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}


In [35]:
best_xgb = XGBClassifier(colsample_bytree= 0.8, gamma= 0, learning_rate=0.01, max_depth= 3, n_estimators = 100)
best_xgb.fit(train_X, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.01, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

### 오차행렬

In [37]:
from dmba import classificationSummary, gainsChart, liftChart

In [38]:
classificationSummary(train_y, best_xgb.predict(train_X))

Confusion Matrix (Accuracy 0.9571)

       Prediction
Actual   0   1
     0   0   9
     1   0 201


In [39]:
classificationSummary(valid_y, best_xgb.predict(valid_X))

Confusion Matrix (Accuracy 0.9333)

       Prediction
Actual  0  1
     0  0  3
     1  0 42


In [40]:
classificationSummary(test_y, best_xgb.predict(test_X))

Confusion Matrix (Accuracy 0.9348)

       Prediction
Actual  0  1
     0  0  3
     1  0 43


# AUC

In [41]:
xgb_param_grid={
    'n_estimators' : [100,200,300,400,500],
    'learning_rate' : [0.01,0.05,0.1,0.15],
    'max_depth' : [3,5,7,10,15],
    'gamma' : [0,1,2,3],
    'colsample_bytree' : [0.8,0.9],
    
}

#여기서 설정 파라미터의 갯수(총 4000개의 조합이므로 4000번의 학습이 돌아감)
xgb_grid=GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="roc_auc", n_jobs=-1, verbose = 2)
xgb_grid.fit(train_X, train_y)

Fitting 5 folds for each of 800 candidates, totalling 4000 fits


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_to_...
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 0.9], 'gamma': [0, 1, 2, 3],
                         'learning_rate': [0.01, 0.05, 0.1, 0.15],
                         'max_depth': [3, 5, 7, 10, 15],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='roc_auc', verbose=2)

In [42]:
#best auc 수치와 best parameter확인
print("best auc : {0: .4f}".format(xgb_grid.best_score_))
print("best param : ",xgb_grid.best_params_)

best acc :  0.7598
best param :  {'colsample_bytree': 0.9, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 200}


In [43]:
best_xgb = XGBClassifier(colsample_bytree=0.9, gamma=0, learning_rate=0.05, max_depth=5, n_estimators=200)
best_xgb.fit(train_X, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.9,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=200,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [44]:
classificationSummary(train_y, best_xgb.predict(train_X))

Confusion Matrix (Accuracy 0.9905)

       Prediction
Actual   0   1
     0   7   2
     1   0 201


In [45]:
classificationSummary(valid_y, best_xgb.predict(valid_X))

Confusion Matrix (Accuracy 0.9333)

       Prediction
Actual  0  1
     0  0  3
     1  0 42


In [46]:
classificationSummary(test_y, best_xgb.predict(test_X))

Confusion Matrix (Accuracy 0.9348)

       Prediction
Actual  0  1
     0  0  3
     1  0 43


## Light GBM의 파라미터에 대해?
#### LGBM의 경우 복잡한 것은 파라미터 튜닝입니다. Light GBM은 100개 이상의 파라미터를 커버하고 있기 때문입니다.

#### Light GBM의 가장 베이직한 파라미터를 아는 것이 구현 시 매우 중요합니다. 

 
### gridsearchCV : 최적 파라미터 선정법 찾기

In [47]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [48]:
from lightgbm import LGBMClassifier

In [49]:
lgbm= LGBMClassifier()

# 정확도

In [51]:
params={
    'lgbm__learging_rate':[0.001,0.01,0.1],
    'lgbm__max_depth':[5,10],
    'lgbm__reg__lambda':[0.1,1],
    'lgbm__subsample':[0.5,1],
    'lgbm__n_estimators':[500,1000]
    }

In [52]:
lgbm_grid=GridSearchCV(lgbm, param_grid = params, scoring="accuracy", n_jobs=5, cv=5)
lgbm_grid.fit(train_X, train_y)

GridSearchCV(cv=5, estimator=LGBMClassifier(), n_jobs=5,
             param_grid={'lgbm__learging_rate': [0.001, 0.01, 0.1],
                         'lgbm__max_depth': [5, 10],
                         'lgbm__n_estimators': [500, 1000],
                         'lgbm__reg__lambda': [0.1, 1],
                         'lgbm__subsample': [0.5, 1]},
             scoring='accuracy')

In [53]:
#best acc 수치와 best parameter확인
print("best acc : {0: .4f}".format(lgbm_grid.best_score_))
print("best param : ",lgbm_grid.best_params_)

best acc :  0.9524
best param :  {'lgbm__learging_rate': 0.001, 'lgbm__max_depth': 5, 'lgbm__n_estimators': 500, 'lgbm__reg__lambda': 0.1, 'lgbm__subsample': 0.5}


In [54]:
best_lgbm = LGBMClassifier(lgbm__learging_rate=0.001, lgbm__max_depth=5, lgbm__n_estimators=500, lgbm__reg__lambda=0.1, lgbm__subsample=0.5)
best_lgbm.fit(train_X, train_y)

LGBMClassifier(lgbm__learging_rate=0.001, lgbm__max_depth=5,
               lgbm__n_estimators=500, lgbm__reg__lambda=0.1,
               lgbm__subsample=0.5)

In [55]:
classificationSummary(train_y, best_lgbm.predict(train_X))

Confusion Matrix (Accuracy 0.9905)

       Prediction
Actual   0   1
     0   7   2
     1   0 201


In [57]:
classificationSummary(valid_y, best_lgbm.predict(valid_X))

Confusion Matrix (Accuracy 0.9333)

       Prediction
Actual  0  1
     0  0  3
     1  0 42


In [58]:
classificationSummary(test_y, best_lgbm.predict(test_X))

Confusion Matrix (Accuracy 0.9130)

       Prediction
Actual  0  1
     0  0  3
     1  1 42


# AUC

In [60]:
params={
    'lgbm__learging_rate':[0.001,0.01,0.1],
    'lgbm__max_depth':[5,10],
    'lgbm__reg__lambda':[0.1,1],
    'lgbm__subsample':[0.5,1],
    'lgbm__n_estimators':[500,1000]
    }

lgbm_grid=GridSearchCV(lgbm, param_grid = params, scoring="roc_auc", n_jobs=5, cv=5)
lgbm_grid.fit(train_X, train_y)

GridSearchCV(cv=5, estimator=LGBMClassifier(), n_jobs=5,
             param_grid={'lgbm__learging_rate': [0.001, 0.01, 0.1],
                         'lgbm__max_depth': [5, 10],
                         'lgbm__n_estimators': [500, 1000],
                         'lgbm__reg__lambda': [0.1, 1],
                         'lgbm__subsample': [0.5, 1]},
             scoring='roc_auc')

In [61]:
#best auc 수치와 best parameter확인
print("best auc : {0: .4f}".format(lgbm_grid.best_score_))
print("best param : ",lgbm_grid.best_params_)

best auc :  0.7195
best param :  {'lgbm__learging_rate': 0.001, 'lgbm__max_depth': 5, 'lgbm__n_estimators': 500, 'lgbm__reg__lambda': 0.1, 'lgbm__subsample': 0.5}


In [62]:
best_lgbm = LGBMClassifier(lgbm__learging_rate=0.001, lgbm__max_depth=5, 
                           lgbm__n_estimators=500, lgbm__reg__lambda=0.1, lgbm__subsample=0.5)
best_lgbm.fit(train_X, train_y)

LGBMClassifier(lgbm__learging_rate=0.001, lgbm__max_depth=5,
               lgbm__n_estimators=500, lgbm__reg__lambda=0.1,
               lgbm__subsample=0.5)

In [63]:
classificationSummary(train_y, best_lgbm.predict(train_X))

Confusion Matrix (Accuracy 0.9905)

       Prediction
Actual   0   1
     0   7   2
     1   0 201


In [64]:
classificationSummary(valid_y, best_lgbm.predict(valid_X))

Confusion Matrix (Accuracy 0.9333)

       Prediction
Actual  0  1
     0  0  3
     1  0 42


In [65]:
classificationSummary(test_y, best_lgbm.predict(test_X))

Confusion Matrix (Accuracy 0.9130)

       Prediction
Actual  0  1
     0  0  3
     1  1 42


## XGBoost
##### - train 정확도 : 0.9571  
##### colsample_bytree= 0.8, gamma= 0, learning_rate=0.01, max_depth= 3, n_estimators = 100
##### - train AUC : 0.7598
##### colsample_bytree=0.9, gamma=0, learning_rate=0.05, max_depth=5, n_estimators=200

## LGBM
##### - train 정확도 : 0.9524
##### lgbm__learging_rate=0.001, lgbm__max_depth=5, lgbm__n_estimators=500, lgbm__reg__lambda=0.1, lgbm__subsample=0.5
##### - train AUC : 0.7195
##### lgbm__learging_rate=0.001, lgbm__max_depth=5, lgbm__n_estimators=500, lgbm__reg__lambda=0.1, lgbm__subsample=0.5

